In [1]:
from twfinch import FINCH
import numpy as np
import scipy.io

In [2]:
#Provide the .mat feature file

mat_data = scipy.io.loadmat('features1.mat')

# Code starts:

features = mat_data['features']
print("Shape of features : ",features.shape)

Shape of features :  (2403, 1000)


In [3]:
# Use only when Ground Truth can be provided
# Get the maximum possible NMI by varying the cluster size
# Provide the frame numbers where transition is expected (frame number vary from 1 to the last frame)
# to get the frame number if the filename is frames_x.jpg, frame_number = x/sample_rate

transitions = [175,569,1342,1572,2174]

# Code starts 
j = 0
x1 = 0
gt = []
x = features.shape[0]
for transition in transitions:
    x2 = transition
    for l in range(x1,x2):
        gt.append(j)
    j = j+1
    x1 = x2
for k in range(transitions[-1],x):
    gt.append(j)

gt = np.array(gt)


from sklearn.metrics import normalized_mutual_info_score as nmi

max_clust = 0
max_score = 0

for i in range(3,50):
    c, num_clust, req_c = FINCH(features, req_clust=i, tw_finch=True)
    score = nmi(gt, req_c)
    max_score = max(max_score,score)
    if(max_score == score):
        max_clust = i

print('Max NMI Score: {:.2f}'.format(max_score * 100))
print('Number of clusters : ', max_clust)

Max NMI Score: 95.36
Number of clusters :  5


In [4]:
# Provide the required number of clusters in the req_clust field
# Provide the directory in which the relevant frames are stored

c, num_clust, req_c = FINCH(features, req_clust=6, tw_finch=True)
dir_name = 'frames1'


import os
frames = os.listdir(dir_name)
frames = sorted(frames, key=lambda x: os.path.getctime(os.path.join(dir_name, x)))

# print(np.unique(req_c))
# print(c.shape)
# print(req_c.shape)

#req_c returns the label of each frame. Frames beloging to the same label form a cluster

#The below code prints the image number which acts as a transition point

curr_value = req_c[0]
for j in range(0,req_c.shape[0]):
    if(not(curr_value == req_c[j])):
        curr_value = req_c[j]
        print(frames[j])

#Get the NMI score

from sklearn.metrics import normalized_mutual_info_score as nmi
score = nmi(gt, req_c)
print('NMI Score: {:.2f}'.format(score * 100))

frame_5690.jpg
frame_10640.jpg
frame_13420.jpg
frame_15720.jpg
frame_21740.jpg
NMI Score: 89.39
